# General Residual Correction Workflow

In [ ]:
from darts import TimeSeries
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
from darts.models import GaussianProcessFilter
from darts import TimeSeries
from sklearn.gaussian_process.kernels import RBF
from darts.models import BlockRNNModel
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape
import ray
import os
import optuna 

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
df = pd.read_csv()#INPUT_CSV